In [1]:
%pip install anthropic==0.48.0
%pip install beautifulsoup4==4.13.3
%pip install dotenv==0.9.9
%pip install huggingface-hub==0.29.3
%pip install lxml==5.3.2
%pip install matplotlib-inline==0.1.7
%pip install ollama==0.4.7
%pip install openai==1.65.1
%pip install pandas==2.0.3
%pip install regex==2024.11.6
%pip install requests==2.32.3
%pip install rich==13.9.4
%pip install rouge==1.0.1
%pip install safetensors==0.5.3
%pip install scikit-learn==1.3.2
%pip install scipy==1.10.1
%pip install scispacy==0.5.5
%pip install sentencepiece==0.2.0
%pip install spacy==3.7.5
%pip install thinc==8.2.5
%pip install tokenizers==0.20.3
%pip install torch==2.4.1
%pip install tqdm==4.67.1
%pip install transformers==4.46.3
%pip install typer==0.15.2

In [2]:
import anthropic
import re
import os
import json
from dotenv import load_dotenv
import datetime
import pickle
import traceback
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
# from ollama import chatF
from ollama import ChatResponse

## Retrieval

In [3]:
import spacy
import requests
import xml.etree.ElementTree as ET
import time
# import config
def escape_for_json(input_string):
    escaped_string = json.dumps(input_string)
    return escaped_string

# Load environment variables from .env file
load_dotenv()

#Suppress warnings about elasticsearch certificates
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
MINDATE = '2000/01/01'  # TODO: find a better starting date
MAXDATE = '2025/01/01'  # TODO: BioASQ 12b requires us to use PubMed 2024 annual baseline version.

import time
import requests
import xml.etree.ElementTree as ET

# 请替换为你的实际API密钥
API_KEY = "your_key"

def run_elasticsearch_query(sentence, max_length=30, verbose=True):
    """Extract medical entities from a sentence and fetch relevant PubMed articles based on those entities."""
    
    print(sentence)
    
    # Step 1: Formulate PubMed query
    # query_term = ' AND '.join(keyword.replace(' ', '+') for keyword, _ in entities)
    # time.sleep(1)  # Adjust the sleep time if needed
    
    # sentence=sentence.replace(' ', '+')
    sentence = re.sub(r'\("([^"]+)"\)', lambda m: '("'+m.group(1).replace(" ", "+")+'")', sentence)
    sentence = re.sub(
    r'\s*\[(Title/Abstract|MeSH)\]\s*',  # 匹配 [Title/Abstract] 或 [MeSH]
    '',                                   # 替换为空字符串
    sentence
    )
    print("变＋号后:",sentence)
    query_term=sentence
    # Step 2: Query PubMed to get PMIDs
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query_term,
        "retmax": max_length,
        "mindate": MINDATE,
        "maxdate": MAXDATE,
        "api_key": API_KEY  # 添加 API 密钥参数
    }
    max_retries = 5
    for attempt in range(max_retries):
        try:
            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                break  # If the request is successful, break out of the loop
            elif response.status_code == 429:  # Too many requests
                print(f"Too many requests. Retrying after {attempt + 1} seconds...")
                time.sleep((attempt + 1) * 2)  # Exponential backoff (backoff increases with each retry)
            else:
                print(f"Failed to retrieve PMIDs. Status code: {response.status_code}")
                return []  # If response is not 200 or 429, return empty list
        except requests.exceptions.RequestException as e:
            print(f"Request failed with exception: {e}. Retrying...")
            time.sleep((attempt + 1) * 2)  # Exponential backoff

    # If we exhaust retries without success, return empty list
    if response.status_code != 200:
        print("Failed to retrieve PMIDs after maximum retries.")
        return []

    # Step 3: Parse PMIDs
    root = ET.fromstring(response.content)
    pmids = [id_elem.text for id_elem in root.findall('.//IdList/Id')]

    # Step 4: Fetch article details
    efetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pubmed",
        "id": ",".join(pmids),
        "retmode": "xml",
        "api_key": API_KEY  # 添加 API 密钥参数
    }

    response = requests.get(efetch_url, params=params)
    if response.status_code != 200:
        print("Failed to retrieve article details. Status code:", response.status_code)
        return []

    # Step 5: Process XML response
    root = ET.fromstring(response.content)
    results = []

    for article in root.findall('.//PubmedArticle'):
        # Extract PMID
        pmid_elem = article.find('.//PMID')
        pmid = pmid_elem.text if pmid_elem is not None else 'N/A'

        # Extract Title
        title_elem = article.find('.//ArticleTitle')
        title = title_elem.text if title_elem is not None else 'No title available'

        # Extract Abstract
        abstract_elem = article.find('.//Abstract')
        abstract_parts = []
        if abstract_elem is not None:
            for text_elem in abstract_elem.iterfind('.//AbstractText'):
                if text_elem.text:
                    abstract_parts.append(text_elem.text.strip())
        abstract = ' '.join(abstract_parts) if abstract_parts else 'No abstract available'
        

        # Build result dict (保持相同的结构)
        result = {
            "id": f"http://www.ncbi.nlm.nih.gov/pubmed/{pmid}",  # 保持相同的URL格式
            "title": title,
            "abstract": abstract  # 合并为单个字符串
        }
        results.append(result)
        # print(results)
        if verbose:
            print(f"Processed PMID: {pmid}")
            print(f"Title: {title}")
            print(f"Abstract: {abstract[:100]}...\n{'-'*80}")

    print(f"docs found: {len(results)}")
    return results  # 返回与第一个函数完全相同的结构


def createQuery(query_string: str, size=50): 
    query = {
        "query": {
            "query_string": {
                "query": query_string
            }
        },
        "size": size
    }
    return query

## Query Expansion

In [4]:
from openai import OpenAI
def expand_query_few_shot(df_prior, n, question:str, model:str):
    messages = generate_n_shot_examples_expansion(df_prior, n)
    # Add the user message
    user_message = {
    "role": "user",
    "content": f"""
    Given a biomedical question, generate a PubMed query string. 
    
    **必须遵守的格式规则：**
    **1. 最终结果必须严格包裹在##之间**
    **2. 结果中不得包含任何解释性文字**
    **3. 必须使用PubMed标准语法**
    
    PubMed语法规则：
    - 使用括号分组术语，用AND/OR连接
    - 用双引号包裹短语（如"muscle weakness"）
    - 支持字段限定（如[Title/Abstract]）
    
    示例（注意格式）：
    **问题：** 维生素D缺乏的影响？
    **正确输出：** ##("vitamin d deficiency"[MeSH] OR "hypovitaminosis d"[Title/Abstract]) AND ("human health"[Title/Abstract] OR "physiological effects"[MeSH])##
    
    请为以下问题生成查询（仅输出##包裹的最终结果）：
    '{question}'
    """
    }
    messages.append(user_message)
    
    print("Prompt Messages:")
    print(messages)

    client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="lm-studio")
    # 调用API生成对话
    completion = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        temperature=0.0
    )
    # client = OpenAI(api_key="your_key", base_url="https://api.deepseek.com")
    # completion = client.chat.completions.create(
    #     model="deepseek-reasoner",
    #     messages=messages,
    #     temperature=0.0
    # )
    answer = completion.choices[0].message.content
    print("\n Completion:")
    print(answer)
    print("\n")
    return answer

def expand_query_wiki(wiki_context: str, question:str, model: str)-> str:
    # Add the user message
    user_message = {
        "role": "user",
        "content": f"""
        {wiki_context}
        Answer this question: '{question}' 
        Think step by step and write an exhaustive answer explaining your reasoning"""
    }
    messages = [
        {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, research, and information retrieval in the biomedical domain."},
        user_message
    ]
    print("\nMessages Expand Query:")
    print(messages)
    client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="lm-studio")
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0 # randomness of completion
    )
    # client = OpenAI(api_key="your_key", base_url="https://api.deepseek.com")
    # completion = client.chat.completions.create(
    #     model="deepseek-reasoner",
    #     messages=messages,
    #     temperature=0.0
    # )
    query = f"""
        {{
            "query": {{
                "more_like_this" : {{
                "fields" : ["title", "abstract"],
                "like" : {escape_for_json(completion.choices[0].message.content)},
                "min_term_freq" : 1,
                "min_doc_freq": 1,
                "boost_terms": 1
                }}
            }},
        "size":100
        }}
    """
    return query
def escape_for_json(text: str) -> str:
    """
    Escape the content for JSON compatibility.
    """
    return text.replace("\"", "\\\"").replace("\n", " ")

def generate_n_shot_examples_expansion(df, n):
    
    # Initialize the system message
    system_message = {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, research, and information retrieval in the biomedical domain."}
    
    # Initialize the list of messages with the system message
    messages = [system_message]
    
    
    if n< 1:
        top_entries = pd.DataFrame()
    else:
        top_entries = df.sort_values(by='f1_score', ascending=False).head(n)
    
    # Loop through each of the top n entries and add the user and assistant messages
    for _, row in top_entries.iterrows():
        question = row['question_body']
        completion = row['completion']
        
        # Replace problematic characters in question
        question = question.replace("/", "\\\\/")
        
        # Add the user message
        user_message = {
        "role": "user",
        "content": f"""
        Given a biomedical question, generate an PubMed query string that incorporates synonyms and related terms to improve the search results while maintaining precision and relevance to the original question. The query should be formatted for PubMed search, which supports the following syntax:

        The query string should:
        - Use parentheses to group terms, and connect terms with 'AND', 'OR', or 'NOT'.
        - Use '""' for exact phrase matching (e.g., "quick brown").
        - Use ':' to specify fields for targeted searches (e.g., title:(quick OR brown)).
        - Include synonyms and related terms to improve recall without introducing irrelevant terms.
        - Ensure the query is specific enough to return relevant results while not being too broad.

        Example format:
        - Question: What are the effects of vitamin D deficiency on the human body?
        - PubMed query string: ("vitamin d" OR "vitamin d3" OR "cholecalciferol") AND (deficiency OR insufficiency OR "low levels") AND ("effects" OR "impact" OR "consequences") AND ("human body" OR "human health")

        Example of PubMed query:
        - ("concizumab" OR "concizumab treatment") AND ("disease" OR "illness" OR "condition" OR "ailment") AND ("indications" OR "application" OR "efficacy" OR "benefit")

        Tips:
        - Focus on the main concepts and entities in the question.
        - Use synonyms and related terms to capture variations in terminology.
        - Use 'AND', 'OR', 'NOT' to combine terms effectively.
        - Ensure the query string is valid for PubMed search by using correct syntax.

        Please generate a PubMed query string for the following biomedical question, and make sure to wrap the final query in ## tags:
        '{question}'
        """
        }
        
        # Add the assistant message
        assistant_message = {
            "role": "assistant",
            "content": completion  
        }
        
        messages.extend([user_message, assistant_message])

    return messages

## Query Refinement

In [5]:
def refine_query_with_no_results(question, original_query, model):
    messages = [
{"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, research, and information retrieval in the biomedical domain."},
{"role": "user", "content": f"""Given that the following search query has returned no documents, please generate a broader query that retains the original question's context and relevance. Return only the query that can directly be used without any explanation text. Focus on maintaining the query's precision and relevance to the original question.

To generate a broader query, consider the following:

Identify the main concepts in the original query and prioritize them based on their importance to the question.
Simplify the query by removing less essential terms or concepts that might be too specific or restrictive.
Use more general terms or synonyms for the main concepts to expand the search scope while maintaining relevance.
Reduce the number of Boolean operators (AND, OR) to make the query less restrictive.
If the original query includes specific drug names, genes, or proteins, consider using their classes or families instead.
Avoid using too many search fields or specific phrases in quotes, as they can limit the search results.
Original question: '{question}', Original query that returned no results: '{original_query}' think step by step an wrapp the improved query in ## tags:"""}
]

    print("Prompt Messages:")
    print(messages)

    # 调用API生成对话
    client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="lm-studio")
    completion = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        temperature=0.0
    )
    # client = OpenAI(api_key="your_key", base_url="https://api.deepseek.com")
    # completion = client.chat.completions.create(
    #     model="deepseek-reasoner",
    #     messages=messages,
    #     temperature=0.0
    # )
    answer = completion.choices[0].message.content
    print("\n Completion:")
    print(answer)
    print("\n")
    return answer
def no_filtered_articles(question, relevant_articles,original_query, model):
    messages = [
    {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, research, and information retrieval in the biomedical domain."},
    {"role": "user", "content": f"""The current query has failed to retrieve articles containing relevant text snippets matching the question context. Generate a revised PubMed query that better targets the actual content needed to answer the question. Return only the executable query wrapped in ## without explanations.

When reconstructing the query consider these critical adjustments:
1. Analyze mismatches between query terms and actual article content
2. Expand coverage of implicit concepts in the question
3. Include symptom/disease/treatment terminology variants
4. Adjust Boolean logic to bridge vocabulary gaps
5. Add contextually related biomedical entities
6. Maintain clinical relevance while increasing lexical coverage

Evidence of failure:
- 0 relevant snippets found in {len(relevant_articles)} filtered articles
- Snippet matching failed for question: '{question}'

Original question: '{question}'
Original failed query: '{original_query}'

Generate improved query:"""}
]

    print("Prompt Messages:")
    print(messages)
    client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="lm-studio")
    completion = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        temperature=0.0
    )
    # client = OpenAI(api_key="your_key", base_url="https://api.deepseek.com")
    # completion = client.chat.completions.create(
    #     model="deepseek-reasoner",
    #     messages=messages,
    #     temperature=0.0
    # )
    answer = completion.choices[0].message.content
    print("\n Completion:")
    print(answer)
    print("\n")
    return answer


## Snippet Extraction

In [6]:
import re
import json
import logging
from regex import compile as regex_compile

# 配置日志
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# 预编译正则表达式（增强版）
CODE_BLOCK_REGEX = regex_compile(
    r'```(?:json)?\s*(\{.*?\}|\[.*?\])\s*```',  # 支持数组
    re.DOTALL
)
JSON_REGEX = regex_compile(
    r'(?<!\\)(?:\\{2})*(?:{(?:[^{}]|(?R))*}|\[(?:[^\[\]]|(?R))*\])',  # 支持数组和对象
    re.DOTALL | re.VERBOSE
)

def find_extract_json(text: str) -> str:
    """超强防御版 JSON 提取器"""
    text = text.strip()
    if not text:
        logger.debug("输入文本为空")
        return ""

    # 阶段1: 提取代码块内容
    code_block_matches = CODE_BLOCK_REGEX.findall(text)
    for match in code_block_matches:
        candidate = match.strip()
        logger.debug(f"发现代码块候选: {candidate[:100]}...")
        if validate_json(candidate):
            logger.info(f"有效代码块 JSON: {candidate[:50]}...")
            return candidate

    # 阶段2: 深度扫描全文
    candidates = []
    for match in JSON_REGEX.finditer(text):
        candidate = match.group().strip()
        logger.debug(f"发现全文候选: {candidate[:100]}...")
        if validate_json(candidate):
            candidates.append(candidate)

    # 选择策略: 优先结构完整的最短候选（长候选更可能包含无效内容）
    if candidates:
        logger.info(f"发现 {len(candidates)} 个有效候选")
        return sorted(
            candidates,
            key=lambda x: (x.count('"'), -len(x)),  # 优先双引号合法数量
            reverse=True
        )[0]
    
    # 阶段3: 终极容错模式
    logger.warning("未找到有效 JSON，尝试容错解析")
    return force_extract_json(text)

def validate_json(candidate: str) -> bool:
    """增强 JSON 验证"""
    try:
        parsed = json.loads(candidate)
        # 验证必须包含 snippets 字段（根据业务需求）
        if isinstance(parsed, dict) and 'snippets' in parsed:
            # 检查 snippets 是否为列表
            if not isinstance(parsed['snippets'], list):
                logger.debug(f"无效 snippets 类型: {type(parsed['snippets'])}")
                return False
            return True
        return False
    except Exception as e:
        logger.debug(f"验证失败: {str(e)}")
        return False

def force_extract_json(text: str) -> str:
    """暴力提取模式"""
    # 尝试提取类似 JSON 的结构
    brackets = []
    buffer = []
    in_string = False
    escape = False
    
    for char in text:
        if char in ('{', '[') and not in_string:
            brackets.append(char)
            buffer.append(char)
        elif char in ('}', ']') and not in_string:
            if not brackets:
                continue
            brackets.pop()
            buffer.append(char)
        elif char == '"':
            if not escape:
                in_string = not in_string
            buffer.append(char)
        elif char == '\\':
            escape = not escape
        else:
            escape = False
            buffer.append(char)
    
    candidate = ''.join(buffer)
    logger.debug(f"暴力提取结果: {candidate[:200]}...")
    return candidate if candidate else ""

from unicodedata import normalize
def normalize_unicode_string(s, form='NFKC'):
    normalized  = normalize('NFKD', s).encode('ascii','ignore').decode()
    return normalized


def generate_n_shot_examples_extraction(examples, n):
    """Takes the top n examples, flattens their messages into one list, and filters out messages with the role 'system'."""
    n_shot_examples = []
    for example in examples[:n]:
        for message in example['messages']:
            if message['role'] != 'system':  # Only add messages that don't have the 'system' role
                n_shot_examples.append(message)
    return n_shot_examples
def clean_ideal_answer(text):
    # 使用正则表达式删除<think>标签及其内容
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    # 去除可能残留的换行符并重新整理格式
    cleaned = re.sub(r'\n{2,}', '\n\n', cleaned.strip())
    return cleaned

def extract_relevant_snippets_few_shot(examples, n, article:str, question:str, model:str) -> str:
    
    system_message = {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, research, and information retrieval in the biomedical domain."}
    messages = [system_message]
    few_shot_examples = generate_n_shot_examples_extraction(examples, n)
    messages.extend(few_shot_examples)
    user_message = {"role": "user", "content": f"""Given this question: '{question}' extract relevant sentences or longer snippets from the following article that help answer the question. 
If no relevant information is present, return an empty array. Return the extracted snippets as a json string array called 'snippets'. ```{article}```"""}
    messages.append(user_message)
    print("Prompt Messages:")
    print(messages)
    

    # 调用API生成对话
    client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="lm-studio")
    completion = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        temperature=0.0
    )
    # client = OpenAI(api_key="your_key", base_url="https://api.deepseek.com")
    # completion = client.chat.completions.create(
    #     model="deepseek-reasoner",
    #     messages=messages,
    #     temperature=0.0
    # )
    answer =completion.choices[0].message.content
    answer=clean_ideal_answer(answer)
    print("\n Completion:")
    print(answer)
    print("\n")
    if hasattr(completion, 'choices'):
        json_response = find_extract_json(answer)
    else:
        json_response = find_extract_json(completion.content[0].text)
    try:
        if isinstance(json_response, str):
            sentences = json.loads(json_response)  # 如果是字符串，则解析为字典
        elif isinstance(json_response, dict):
            sentences = json_response  # 如果是字典，则直接使用
        else:
            print(f"Unexpected json_response type: {type(json_response)}")
            sentences = {"snippets": []}
    except Exception as e:
        print(f"Error parsing response as json: {json_response}: {e}")
        traceback.print_exc()
        sentences = {"snippets": []}
    
    print("sentences为",sentences)
    snippets = generate_snippets_from_sentences(article, sentences['snippets'])
    
    return snippets

def find_offset_and_create_snippet(document_id, text, sentence, section):
    text = normalize_unicode_string(text)
    sentence = normalize_unicode_string(sentence)
    offset_begin = text.find(sentence)
    offset_end = offset_begin + len(sentence)
    return {
        "document": document_id,
        "offsetInBeginSection": offset_begin,
        "offsetInEndSection": offset_end,
        "text": sentence,
        "beginSection": section,
        "endSection": section
    }

def generate_snippets_from_sentences(article, sentences):
    snippets = []
        # 确保字段存在且为字符串
    article_abstract = str(article.get('abstract', ''))  # 强制转换为字符串
    article_title = str(article.get('title', ''))       # 即使字段不存在也返回空字符串
        # 处理规范化
    article_abstract = normalize_unicode_string(article_abstract)
    article_title = normalize_unicode_string(article_title)
    # article_abstract = article.get('abstract') or ''  # This will use '' if 'abstract' is None or does not exist
    # article_abstract = normalize_unicode_string(article_abstract)
    # article_title = normalize_unicode_string(article.get('title'))

    for sentence in sentences:
        sentence = normalize_unicode_string(sentence)
        if sentence in article_title:
            snippet = find_offset_and_create_snippet(article['id'], article['title'], sentence, "title")
            snippets.append(snippet)
        elif sentence in article_abstract:
            snippet = find_offset_and_create_snippet(article['id'], article_abstract, sentence, "abstract")
            snippets.append(snippet)
        else:
            print("\nsentences not found in article: "+sentence+"\n")
            print(article)

    return snippets

## Snippet Reranking

In [7]:
def generate_n_shot_examples_reranking(examples, n):
    """Takes the top n examples, flattens their messages into one list, and filters out messages with the role 'system'."""
    n_shot_examples = []
    for example in examples[:n]:
        for message in example['messages']:
            if message['role'] != 'system':  # Only add messages that don't have the 'system' role
                n_shot_examples.append(message)
    return n_shot_examples
def clean_ideal_answer(text):
    # 使用正则表达式删除<think>标签及其内容
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    # 去除可能残留的换行符并重新整理格式
    cleaned = re.sub(r'\n{2,}', '\n\n', cleaned.strip())
    return cleaned
def rerank_snippets(examples, n, snippets, question:str, model:str) -> str:
    numbered_snippets = [{'id': idx, 'text': snippet['text']} for idx, snippet in enumerate(snippets)]
    system_message = {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, research, and information retrieval in the biomedical domain."}
    messages = [system_message]
    few_shot_examples = generate_n_shot_examples_reranking(examples, n)
    messages.extend(few_shot_examples)
    user_message = {"role": "user", "content": f"""Given this question: '{question}' select the top 10 snippets that are most helpfull for answering this question from
                    this list of snippets, rerank them by helpfullness: ```{numbered_snippets}``` return a json array of their ids called 'snippets'"""}
    messages.append(user_message)
    print("Prompt Messages:")
    print(messages)
    
    # 调用API生成对话
    client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="lm-studio")
    completion = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        temperature=0.0
    )
    # client = OpenAI(api_key="your_key", base_url="https://api.deepseek.com")
    # completion = client.chat.completions.create(
    #     model="deepseek-reasoner",
    #     messages=messages,
    #     temperature=0.0
    # )
    answer =completion.choices[0].message.content
    answer=clean_ideal_answer(answer)
    print("\n Completion:")
    print(answer)
    print("\n")
    if hasattr(completion, 'choices'):
        json_response = find_extract_json(answer)
    else:
        json_response = find_extract_json(completion.content[0].text)
    
    try:
        snippets_reranked = json.loads(json_response)
        # snippets_reranked = json_response
        snippets_idx = snippets_reranked['snippets']
        filtered_array = [snippets[i] for i in snippets_idx]
    except Exception as e:
        print(f"Error parsing response as json: {json_response}: {e}")
        traceback.print_exc()
        filtered_array = snippets
        
    return filtered_array

## Run

In [ ]:
import datetime
import time
model_name = "deepSeek-r1:32b"
model_name_extract = "deepSeek-r1:32b"
model_name_rerank = "deepSeek-r1:32b"

n_shot = 10                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

# Get the current timestamp in a sortable format
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if '/' in model_name or ':' in model_name:
    pickl_name = model_name.replace('/', '-').replace(':', '-')
else:
    pickl_name = model_name
pickl_file = f'{pickl_name}-{n_shot}-shot.pkl'

def save_state(data, file_path=pickl_file):
    """Save the current state to a pickle file."""
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

def load_state(file_path=pickl_file):
    """Load the state from a pickle file if it exists, otherwise return None."""
    try:
        if os.path.exists(file_path):
            with open(file_path, 'rb') as f:
                return pickle.load(f)
    except EOFError:  # Handles empty pickle file scenario
        return None
    return None

def read_jsonl_file(file_path):
    """Reads a JSONL file and returns a list of examples."""
    examples = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            examples.append(json.loads(line))
    return examples

def extract_text_wrapped_in_tags(input_string):
    # pattern = "##(.*?)##"
    pattern = r"##(.*?)##|```(.*?)```"
    match = re.search(pattern, input_string, re.DOTALL)  
    if match:
        # Remove line breaks from the matched string
        extracted_text = match.group(1).replace('\n', '')
        return extracted_text
    else:
        return "ERROR"

def reorder_articles_by_snippet_sequence(relevant_article_ids, snippets):
    ordered_article_ids = []
    mentioned_article_ids = set()

    # Add article IDs in the order they appear in the snippets
    for snippet in snippets:
        document_id = snippet['document']
        if document_id in relevant_article_ids and document_id not in mentioned_article_ids:
            ordered_article_ids.append(document_id)
            mentioned_article_ids.add(document_id)

    # Add the remaining article IDs that weren't mentioned in snippets
    for article_id in relevant_article_ids:
        if article_id not in mentioned_article_ids:
            ordered_article_ids.append(article_id)

    return ordered_article_ids


def get_relevant_snippets(examples, n, articles, question, model_name):
    processed_articles = []
    for article in articles:
        time.sleep(1)
        snippets = extract_relevant_snippets_few_shot(examples, n, article, question, model_name)
        print("snippets为",snippets)
        if snippets:
            article['snippets'] = snippets
            processed_articles.append(article)
    return processed_articles

# Run specific few-shot configuration
query_examples = pd.read_csv('2024-03-26_19-24-27_claude-3-opus-20240229_11B1-10-Shot_Retrieval.csv')

snip_extract_examples_file = "Snippet_Extraction_Examples.jsonl"     
snip_extract_examples = read_jsonl_file(snip_extract_examples_file)

snip_rerank_examples_file = "Snippet_Reranking_Examples.jsonl"     
snip_rerank_examples = read_jsonl_file(snip_rerank_examples_file)

def clean_ideal_answer(text):
    # 使用正则表达式删除<think>标签及其内容
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    # 去除可能残留的换行符并重新整理格式
    cleaned = re.sub(r'\n{2,}', '\n\n', cleaned.strip())
    return cleaned


def process_question(question):
    try:
        query_string = ""
        improved_query_string = ""
        relevant_articles_ids = []
        filtered_articles_ids = [] 
        reordered_articles_ids = []
        relevant_snippets = []

        question_id = question['id']
        print(f"Processing question {question_id}")
        wiki_context = ""

        #0 query expansion
        completion = expand_query_few_shot(query_examples, n_shot, question['body'], model_name)
        clean_completion=clean_ideal_answer(completion)
        query_string = extract_text_wrapped_in_tags(clean_completion)
        # print('------------------------------------------------')
        # print(query_string)
        # print('------------------------------------------------')
        # query = createQuery(query_string)
        query = query_string
        print(query)
        print(question['body'])
        relevant_articles = run_elasticsearch_query(query)
        # relevant_articles = run_elasticsearch_query(question['body'])
        if len(relevant_articles) == 0:
            improved_query_completion = refine_query_with_no_results(question['body'], query_string, model_name)
            improved_query_completion=clean_ideal_answer(improved_query_completion)
            improved_query_string = extract_text_wrapped_in_tags(improved_query_completion)
            # query = createQuery(improved_query_string)
            query = improved_query_string
            relevant_articles = run_elasticsearch_query(query)
            # relevant_articles = run_elasticsearch_query(question['body'])
            if len(relevant_articles) > 0:
                print("query refinement worked")
        
        relevant_articles_ids = [article['id'] for article in relevant_articles]
        
        #1 snippet extraction
        filtered_articles = get_relevant_snippets(snip_extract_examples, n_shot, relevant_articles, question['body'], model_name_extract)
        # if len(filtered_articles) == 0:
        #     pro_improved_query_completion = no_filtered_articles(question['body'], relevant_articles,query_string, model_name)
        #     pro_improved_query_completion=clean_ideal_answer(pro_improved_query_completion)
        #     pro_improved_query_string = extract_text_wrapped_in_tags(pro_improved_query_completion)
        #     # query = createQuery(improved_query_string)
        #     query = pro_improved_query_string
        #     relevant_articles = run_elasticsearch_query(query)
        #     relevant_articles_ids = [article['id'] for article in relevant_articles]
        #     filtered_articles = get_relevant_snippets(snip_extract_examples, n_shot, relevant_articles, question['body'], model_name_extract)
        max_iterations = 2  # 设置最大循环次数
        iterations = 0  # 初始化循环次数计数器
        
        while len(filtered_articles) <1 and iterations < max_iterations:
            improved_query_completion =refine_query_with_no_results(question['body'], query, model_name)
            improved_query_completion=clean_ideal_answer(improved_query_completion)
            improved_query_string = extract_text_wrapped_in_tags(improved_query_completion)
            # query = createQuery(improved_query_string)
            query = improved_query_string
            relevant_articles = run_elasticsearch_query(query)
            # relevant_articles = run_elasticsearch_query(question['body'])
            relevant_articles_ids = [article['id'] for article in relevant_articles]
            filtered_articles = get_relevant_snippets(snip_extract_examples, n_shot, relevant_articles, question['body'], model_name_extract)
            iterations += 1  # 增加循环次数
            if len(filtered_articles) >=1:
                print("filtered_articles have one!")
        filtered_articles_ids = [article['id'] for article in filtered_articles]
        relevant_snippets = [snippet for article in filtered_articles for snippet in article['snippets']]
        print("relevant_snippets:",relevant_snippets)
        
    
        #2 rerank snippets
        reranked_snippets = rerank_snippets(snip_rerank_examples, n_shot, relevant_snippets, question['body'], model_name_rerank)
        
        reordered_articles_ids = reorder_articles_by_snippet_sequence(filtered_articles_ids, reranked_snippets)

        return {
            "question_id": question["id"],
            "question_body": question["body"],
            "question_type": question["type"],
            "wiki_context": wiki_context,
            "completion": completion,
            "query": query_string,
            "improved_query": improved_query_string,
            "relevant_articles": relevant_articles_ids,
            "filtered_articles": filtered_articles_ids,
            "documents": reordered_articles_ids,
            "snippets": reranked_snippets
        }
    except Exception as e:
        print(f"Error processing question {question['id']}: {e}")
        traceback.print_exc()
        return {
            "question_id": question.get("id", "error"),
            "question_body": question.get("body", "error"),
            "question_type": question.get("type", "error"),
            "query": query_string or "error",
            "improved_query": improved_query_string or "error",
            "relevant_articles": relevant_articles_ids or [],
            "filtered_articles": filtered_articles_ids or [],
            "documents": reordered_articles_ids[:10] if reordered_articles_ids else [],
            "snippets": relevant_snippets or []
        }

# Define columns
columns = ['question_id', 'question_body', 'question_type', 'wiki_context', 'completion', 'query', 'improved_query', 'relevant_articles', 'filtered_articles', 'documents', 'snippets']

# Initialize empty DataFrame
questions_df = pd.DataFrame(columns=columns)

# Load the input file in JSON format
input_file_name = '/workspace/code/bioasq2024/02_12B/Batch1/PhaseA/BioASQ-task13bPhaseA-testset1.json'


with open(input_file_name) as input_file:
    data = json.loads(input_file.read())

# Assuming 'load_state' returns a DataFrame or None
saved_df = load_state(pickl_file)

if saved_df is not None and not saved_df.empty:
    processed_ids = set(saved_df['question_id'])  # Assuming 'question_id' is your identifier
    questions_df = saved_df
else:
    processed_ids = set()

# Assuming `data["questions"]` is your list of questions to process
# Filter out questions that have already been processed
questions_to_process = [q for q in data["questions"] if q["id"] not in processed_ids]
#questions_to_process = questions_to_process[:2]


# Use ThreadPoolExecutor to process questions in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    # Dictionary to keep track of question futures
    future_to_question = {executor.submit(process_question, q): q for q in questions_to_process}
    
    for future in as_completed(future_to_question):
        question = future_to_question[future]
        try:
            result = future.result()
            if result:
                # Append result to the DataFrame
                result_df = pd.DataFrame([result])
                questions_df = pd.concat([questions_df, result_df], ignore_index=True)
                save_state(questions_df, pickl_file)
        except Exception as e:
            print(f"Error processing question {question['id']}: {e}")
            traceback.print_exc()


# Prefix the output file name with the timestamp
if '/' in model_name:
    model_name_pretty = model_name.split("/")[-1]
else:
    model_name_pretty = model_name
output_file_name = f"./Results/{timestamp}_{model_name_pretty}_2024AB1-Fine-Tuned-{n_shot}-Shot.csv"

# Ensure the directory exists before saving
os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

questions_df.to_csv(output_file_name, index=False)

# After processing all questions and saving the final output:
try:
    # Check if the pickle file exists before attempting to delete it
    if os.path.exists(pickl_file):
        os.remove(pickl_file)
        print("Intermediate state pickle file deleted successfully.")
except Exception as e:
    print(f"Error deleting pickle file: {e}")
    traceback.print_exc()

## Create Run File

In [ ]:
import pandas as pd
import json



def csv_to_json(csv_filepath, json_filepath):
    empty = 0
    # Step 1: Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_filepath)
    
    # Transform the DataFrame into a list of dictionaries, one per question
    questions_list = df.to_dict(orient='records')
    
    # Initialize the structure of the JSON file
    json_structure = {"questions": []}
    
    # Step 2: Transform the DataFrame into the desired JSON structure
    for item in questions_list:
        # Adjusting exact_answer format based on question_type
        if item["question_type"] in ["list", "factoid"]:
            exact_answer_format = [[]]  # For 'list' or 'factoid', it's a list of lists
        else:
            exact_answer_format = ""  # Default to an empty string
            
            
        if len(eval(item["relevant_articles"])) == 0:
            empty = empty +1
        #print(len(eval(item["relevant_articles"])))
        # Construct question_dict conditionally excluding 'exact_answer' for 'ideal' type
        question_dict = {
            "documents": eval(item["documents"])[:10],
            "snippets": eval(item["snippets"])[:10],
            "body": item["question_body"],
            "type": item["question_type"],
            "id": item["question_id"],
            "ideal_answer": ""
        }
        if item["question_type"] != "summary":
            question_dict["exact_answer"] = exact_answer_format
        
        json_structure["questions"].append(question_dict)
    
    # Step 3: Write the JSON structure to a file
    with open(json_filepath, 'w', encoding='utf-8') as json_file:
        json.dump(json_structure, json_file, ensure_ascii=False, indent=4)
    print(empty)

# Example usage
csv_filepath = '/workspace/code/bioasq2024/02_12B/Batch1/PhaseA/Results/2025-05-07_16-30-34_deepSeek-r1:32b_2024AB1-Fine-Tuned-1-Shot.csv'  # Update this path to your actual CSV file path
json_filepath = '/workspace/code/bioasq2024/02_12B/Batch1/PhaseA/Results/bioasq13b-batch4-deepseek-32b.json'  # Update this path to where you want to save the JSON file
csv_to_json(csv_filepath, json_filepath)


In [ ]:
test_string = '{"snippets": ["In this narrative review, we summarize the current management of HER2-low breast cancer. We highlight the findings of the DESTINY-Breast 04\\xa0phase 3 trial, which confirmed the efficacy of trastuzumab-deruxtecan (T-DXd) for the treatment of patients with advanced, pretreated HER2-low breast cancer."]}'
test_string2 = '{"snippets": ["The limit of detection (LOD) and the limit of quantification (LOQ) for the target DNA could reach 32 pM and 1 nM, respectively."],"additional_properties":{"pubmed_id":"33303163"}'
test_string3 = '{"snippets": ["The limit of blank (LOB) was 3\tµg calprotectin/g faeces (µg/g), LOD 8\tμg/g and LOQ 20\tμg/g."]}'
test_string4 = '{"snippets": ["In this narrative review, we summarize the current management of HER2-low breast cancer. We highlight the findings of the DESTINY-Breast 04\\xa0phase 3 trial, which confirmed the efficacy of trastuzumab-deruxtecan (T-DXd) for the treatment of patients with advanced, pretreated HER2-low breast cancer."]}'
test_string4 = test_string4.replace('\\', "\\\\")
print(test_string4)
json.loads(test_string4)